In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.optimizers import SGD, Adam, Adagrad
from keras import backend as K
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Reshape, Activation, Dropout
from keras.layers import Concatenate, Add
from keras.callbacks import ModelCheckpoint
os.chdir('code')
from utils import tf_haversine,get_clusters
from data import load_data
os.chdir('..')

C:\Users\Mohamed\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = load_data(nrows=100000)

In [3]:
clusters = get_clusters(data.train_labels)

In [4]:
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam, Adagrad
from keras import backend as K
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation, Dropout,Input
from keras.layers.core import Reshape
from keras.layers import Concatenate,Add
from keras.callbacks import ModelCheckpoint
from utils import tf_haversine
from data import load_data
from utils import get_clusters
from keras.layers import concatenate



def start_new_session():
    """
    Starts a new Tensorflow session.
    """
    
    # Make sure the session only uses the GPU memory that it actually needs
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    
    session = tf.Session(config=config, graph=tf.get_default_graph())
    K.tensorflow_backend.set_session(session)


def first_last_k(coords):
    """
    Returns a list with the first k and last k GPS coordinates from the given trip.
    The returned list contains 4k values (latitudes and longitudes for 2k points).
    """
    k = 5
    partial = [coords[0] for i in range(2*k)]
    num_coords = len(coords)
    if num_coords < 2*k:
        partial[-num_coords:] = coords
    else:
        partial[:k] = coords[:k]
        partial[-k:] = coords[-k:]
    partial = np.row_stack(partial)
    return np.array(partial).flatten()


def process_features(df):
    """
    Process the features required by our model from the given dataframe.
    Return the features in a list so that they can be merged in our model's input layer.
    """
    # Fetch the first and last GPS coordinates
    coords = np.row_stack(df['POLYLINE'].apply(first_last_k))
    # Standardize latitudes (odd columns) and longitudes (even columns)
    latitudes = coords[:,::2]
    coords[:,::2] = scale(latitudes)
    longitudes = coords[:,1::2]
    coords[:,1::2] = scale(longitudes)
    
    return [
        df['QUARTER_HOUR'].as_matrix(),
        df['DAY_OF_WEEK'].as_matrix(),
        df['WEEK_OF_YEAR'].as_matrix(),
        df['ORIGIN_CALL_ENCODED'].as_matrix(),
        df['TAXI_ID_ENCODED'].as_matrix(),
        df['ORIGIN_STAND_ENCODED'].as_matrix(),
        coords,
    ]


def create_model(metadata, clusters):
    """
    Creates all the layers for our neural network model.
    """
      
    # Arbitrary dimension for all embeddings
    embedding_dim = 10

    # Quarter hour of the day embedding
    input_quarter_hour = Input(shape=(1,))
    embed_quarter_hour = Embedding(metadata['n_quarter_hours'], embedding_dim, input_length=1)(input_quarter_hour)
    embed_quarter_hour = Reshape((embedding_dim,))(embed_quarter_hour)

    # Day of the week embedding
    input_day_of_week = Input(shape=(1,))
    embed_day_of_week = Embedding(metadata['n_days_per_week'], embedding_dim, input_length=1)(input_day_of_week)
    embed_day_of_week = Reshape((embedding_dim,))(embed_day_of_week)

    # Week of the year embedding
    input_week_of_year = Input(shape=(1,))
    embed_week_of_year = Embedding(metadata['n_weeks_per_year'], embedding_dim, input_length=1)(input_week_of_year)
    embed_week_of_year = Reshape((embedding_dim,))(embed_week_of_year)

    # Client ID embedding
    input_client_ids = Input(shape=(1,))
    embed_client_ids = Embedding(metadata['n_client_ids'], embedding_dim, input_length=1)(input_client_ids)
    embed_client_ids = Reshape((embedding_dim,))(embed_client_ids)


    # Taxi ID embedding
    input_taxi_ids = Input(shape=(1,))
    embed_taxi_ids = Embedding(metadata['n_taxi_ids'], embedding_dim, input_length=1)(input_taxi_ids)
    embed_taxi_ids = Reshape((embedding_dim,))(embed_taxi_ids)


    # Taxi stand ID embedding
    input_stand_ids = Input(shape=(1,))
    embed_stand_ids = Embedding(metadata['n_stand_ids'], embedding_dim, input_length=1)(input_stand_ids)
    embed_stand_ids = Reshape((embedding_dim,))(embed_stand_ids)
    
    # GPS coordinates (5 first lat/long and 5 latest lat/long, therefore 20 values)

    coords_in = Input(shape=(20,))
    
    #model = Sequential()
    
    concatenated = concatenate([
                embed_quarter_hour,
                embed_day_of_week,
                embed_week_of_year,
                embed_client_ids,
                embed_taxi_ids,
                embed_stand_ids,
                coords_in
            ])
    
    out = Dense(500, activation='relu')(concatenated)
    
    out = Dense(len(clusters),activation='softmax',name='output_layer')(out)
    
    cast_clusters = K.cast_to_floatx(clusters)
    def destination(probabilities):
        return tf.matmul(probabilities, cast_clusters)
    
    out = Activation(destination)(out)
    
    model = Model([
                input_quarter_hour,
                input_day_of_week,
                input_week_of_year,
                input_client_ids,
                input_taxi_ids,
                input_stand_ids,
                coords_in
            ],out)
    
    

    # Compile the model
    optimizer = SGD(lr=0.01, momentum=0.9, clipvalue=1.)  # Use `clipvalue` to prevent exploding gradients
    model.compile(loss=tf_haversine, optimizer=optimizer)
    
    return model


def full_train(data,clusters,n_epochs=100, batch_size=200, save_prefix=None):
    """
    Runs the complete training process.
    """
    
    # Load initial data
    print("Loading data...")
    #data = load_data()
    data = data
    
    # Estimate the GPS clusters
    print("Estimating clusters...")
    #clusters = get_clusters(data.train_labels)
    clusters = clusters
    
    # Set up callbacks
    callbacks = []
    if save_prefix is not None:
        # Save the model's intermediary weights to disk after each epoch
        file_path="cache/%s-{epoch:03d}-{val_loss:.4f}.hdf5" % save_prefix
        callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', mode='min', save_weights_only=True, verbose=1))

    # Create model
    print("Creating model...")
    start_new_session()
    model = create_model(data.metadata, clusters)
    
    # Run the training
    print("Start training...")
    history = model.fit(
        process_features(data.train), data.train_labels,
        nb_epoch=n_epochs, batch_size=batch_size,
        validation_data=(process_features(data.validation), data.validation_labels),
        callbacks=callbacks)

    if save_prefix is not None:
        # Save the training history to disk
        file_path = 'cache/%s-history.pickle' % save_prefix
        with open(file_path, 'wb') as handle:
            pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    return history

In [ ]:
os.chdir('code')
from utils import tf_haversine,get_clusters
os.chdir('..')
full_train(data,clusters)

Loading data...
Estimating clusters...
Creating model...
Start training...


C:\Users\Mohamed\Anaconda3\lib\site-packages\ipykernel_launcher.py:192: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 94831 samples, validate on 968 samples
Epoch 1/100
94831/94831 [==============================] - 95s 999us/step - loss: 3.0312 - val_loss: 2.6045
Epoch 2/100
94831/94831 [==============================] - 93s 984us/step - loss: 2.2222 - val_loss: 2.0110
Epoch 3/100
94831/94831 [==============================] - 96s 1ms/step - loss: 1.8971 - val_loss: 1.7752
Epoch 4/100
65600/94831 [===================>..........] - ETA: 28s - loss: 1.8159